In [137]:
# Dependencies
import requests
import json
import quandl
import pandas as pd
from datetime import datetime


In [138]:
date_df = pd.read_csv('../../db/sentences_sample_with_datetime2.csv',sep=',')
date_df['call_date'] = pd.to_datetime(date_df['call_date']).dt.date
date_df['base_date'] = pd.to_datetime(date_df['base_date']).dt.date
date_df['next_date'] = pd.to_datetime(date_df['next_date']).dt.date

date_df.head()



,call_datetime,base_date,base_day_of_week,call_date,call_section,call_time,call_title,id,next_date,sentence,sentiment,speaker,ticker
0,2019-10-29 10:00:00,2019-10-28,0.0,2019-10-29,question,10:00:00,General Motors Company (NYSE:GM) Q3 2019 Earni...,1833,2019-10-29,Any update that you can share in terms of mile...,NaN,Itay Michaeli,GM
1,2019-10-24 08:00:00,2019-10-23,2.0,2019-10-24,answer,08:00:00,"Twitter, Inc. (NYSE:TWTR) Q3 2019 Results Earn...",3984,2019-10-24,Hope that helps.,NaN,Ned Segal,TWTR
2,2019-10-24 08:00:00,2019-10-23,2.0,2019-10-24,answer,08:00:00,"Twitter, Inc. (NYSE:TWTR) Q3 2019 Results Earn...",3929,2019-10-24,Since you're probably wondering and we've talk...,NaN,Ned Segal,TWTR
3,2019-10-30 08:00:00,2019-10-29,1.0,2019-10-30,presentation,08:00:00,General Electric Company (NYSE:GE) Q3 2019 Ear...,1004,2019-10-30,Good morning and welcome to GE's third quarter...,NaN,Steve Winoker,GE
4,2019-10-29 10:00:00,2019-10-28,0.0,2019-10-29,answer,10:00:00,General Motors Company (NYSE:GM) Q3 2019 Earni...,2010,2019-10-29,There's no other like one time item or whateve...,NaN,Dhivya Suryadevara,GM


In [139]:
price_df = pd.read_csv('../../db/ticker_closing_prices.csv',sep=',')
price_df['Date'] = pd.to_datetime(price_df['Date']).dt.date
price_df.head()


,Date,High,Low,Open,Close,Volume,Adj Close,ticker
0,2019-01-02,17.040001,16.320000,16.420000,16.840000,5222300.0,15.740013,IVZ
1,2019-01-03,16.850000,16.360001,16.750000,16.430000,6000500.0,15.356794,IVZ
2,2019-01-04,17.340000,16.620001,16.750000,17.260000,5041500.0,16.132580,IVZ
3,2019-01-07,17.719999,17.010000,17.190001,17.469999,3884700.0,16.328859,IVZ
4,2019-01-08,17.820000,17.190001,17.600000,17.280001,3998100.0,16.151274,IVZ


In [140]:
merged_df = date_df
merged_df.head()

,call_datetime,base_date,base_day_of_week,call_date,call_section,call_time,call_title,id,next_date,sentence,sentiment,speaker,ticker
0,2019-10-29 10:00:00,2019-10-28,0.0,2019-10-29,question,10:00:00,General Motors Company (NYSE:GM) Q3 2019 Earni...,1833,2019-10-29,Any update that you can share in terms of mile...,NaN,Itay Michaeli,GM
1,2019-10-24 08:00:00,2019-10-23,2.0,2019-10-24,answer,08:00:00,"Twitter, Inc. (NYSE:TWTR) Q3 2019 Results Earn...",3984,2019-10-24,Hope that helps.,NaN,Ned Segal,TWTR
2,2019-10-24 08:00:00,2019-10-23,2.0,2019-10-24,answer,08:00:00,"Twitter, Inc. (NYSE:TWTR) Q3 2019 Results Earn...",3929,2019-10-24,Since you're probably wondering and we've talk...,NaN,Ned Segal,TWTR
3,2019-10-30 08:00:00,2019-10-29,1.0,2019-10-30,presentation,08:00:00,General Electric Company (NYSE:GE) Q3 2019 Ear...,1004,2019-10-30,Good morning and welcome to GE's third quarter...,NaN,Steve Winoker,GE
4,2019-10-29 10:00:00,2019-10-28,0.0,2019-10-29,answer,10:00:00,General Motors Company (NYSE:GM) Q3 2019 Earni...,2010,2019-10-29,There's no other like one time item or whateve...,NaN,Dhivya Suryadevara,GM


In [141]:
merged_df = date_df.merge(price_df, left_on=['ticker','base_date'], right_on = ['ticker','Date'], how='left')
merged_df = merged_df.drop(columns=['Date', 'High','Low','Open','Adj Close'])
merged_df = merged_df.rename(columns={"Close": "base_close", "Volume": "base_volume"})
merged_df.head()

,call_datetime,base_date,base_day_of_week,call_date,call_section,call_time,call_title,id,next_date,sentence,sentiment,speaker,ticker,base_close,base_volume
0,2019-10-29 10:00:00,2019-10-28,0.0,2019-10-29,question,10:00:00,General Motors Company (NYSE:GM) Q3 2019 Earni...,1833,2019-10-29,Any update that you can share in terms of mile...,NaN,Itay Michaeli,GM,36.639999,10645500.0
1,2019-10-24 08:00:00,2019-10-23,2.0,2019-10-24,answer,08:00:00,"Twitter, Inc. (NYSE:TWTR) Q3 2019 Results Earn...",3984,2019-10-24,Hope that helps.,NaN,Ned Segal,TWTR,38.830002,20276200.0
2,2019-10-24 08:00:00,2019-10-23,2.0,2019-10-24,answer,08:00:00,"Twitter, Inc. (NYSE:TWTR) Q3 2019 Results Earn...",3929,2019-10-24,Since you're probably wondering and we've talk...,NaN,Ned Segal,TWTR,38.830002,20276200.0
3,2019-10-30 08:00:00,2019-10-29,1.0,2019-10-30,presentation,08:00:00,General Electric Company (NYSE:GE) Q3 2019 Ear...,1004,2019-10-30,Good morning and welcome to GE's third quarter...,NaN,Steve Winoker,GE,9.070000,45759000.0
4,2019-10-29 10:00:00,2019-10-28,0.0,2019-10-29,answer,10:00:00,General Motors Company (NYSE:GM) Q3 2019 Earni...,2010,2019-10-29,There's no other like one time item or whateve...,NaN,Dhivya Suryadevara,GM,36.639999,10645500.0


In [142]:
merged_df = merged_df.merge(price_df, left_on=['ticker','next_date'], right_on = ['ticker','Date'], how='left')
merged_df = merged_df.drop(columns=['Date', 'High','Low','Open','Adj Close'])
merged_df = merged_df.rename(columns={"Close": "next_close", "Volume": "next_volume"})
merged_df

,call_datetime,base_date,base_day_of_week,call_date,call_section,call_time,call_title,id,next_date,sentence,sentiment,speaker,ticker,base_close,base_volume,next_close,next_volume
0,2019-10-29 10:00:00,2019-10-28,0.0,2019-10-29,question,10:00:00,General Motors Company (NYSE:GM) Q3 2019 Earni...,1833,2019-10-29,Any update that you can share in terms of mile...,NaN,Itay Michaeli,GM,36.639999,10645500.0,38.209999,18562500.0
1,2019-10-24 08:00:00,2019-10-23,2.0,2019-10-24,answer,08:00:00,"Twitter, Inc. (NYSE:TWTR) Q3 2019 Results Earn...",3984,2019-10-24,Hope that helps.,NaN,Ned Segal,TWTR,38.830002,20276200.0,30.750000,105112500.0
2,2019-10-24 08:00:00,2019-10-23,2.0,2019-10-24,answer,08:00:00,"Twitter, Inc. (NYSE:TWTR) Q3 2019 Results Earn...",3929,2019-10-24,Since you're probably wondering and we've talk...,NaN,Ned Segal,TWTR,38.830002,20276200.0,30.750000,105112500.0
3,2019-10-30 08:00:00,2019-10-29,1.0,2019-10-30,presentation,08:00:00,General Electric Company (NYSE:GE) Q3 2019 Ear...,1004,2019-10-30,Good morning and welcome to GE's third quarter...,NaN,Steve Winoker,GE,9.070000,45759000.0,10.110000,215943400.0
4,2019-10-29 10:00:00,2019-10-28,0.0,2019-10-29,answer,10:00:00,General Motors Company (NYSE:GM) Q3 2019 Earni...,2010,2019-10-29,There's no other like one time item or whateve...,NaN,Dhivya Suryadevara,GM,36.639999,10645500.0,38.209999,18562500.0
5,2019-10-30 08:00:00,2019-10-29,1.0,2019-10-30,presentation,08:00:00,General Electric Company (NYSE:GE) Q3 2019 Ear...,1042,2019-10-30,"We started putting that cash to work, with a $...",NaN,Larry Culp,GE,9.070000,45759000.0,10.110000,215943400.0
6,2019-10-15 11:00:00,2019-10-14,0.0,2019-10-15,presentation,11:00:00,Wells Fargo & Company (NYSE:WFC) Q3 2019 Earni...,4837,2019-10-15,We have 30.2 million digital active customers ...,NaN,John Shrewsberry,WFC,49.270000,16870500.0,50.110001,38478400.0
7,2019-10-30 08:00:00,2019-10-29,1.0,2019-10-30,answer,08:00:00,General Electric Company (NYSE:GE) Q3 2019 Ear...,1563,2019-10-30,"So, I think what you see a little bit relative...",NaN,Larry Culp,GE,9.070000,45759000.0,10.110000,215943400.0
8,2019-10-23 09:00:00,2019-10-22,1.0,2019-10-23,answer,09:00:00,Invesco Ltd. (NYSE:IVZ) Q3 2019 Earnings Confe...,2226,2019-10-23,I think it is very hard to provide guidance on...,NaN,Loren Starr,IVZ,16.030001,8068200.0,17.280001,9388100.0
9,2019-10-15 11:00:00,2019-10-14,0.0,2019-10-15,presentation,11:00:00,Wells Fargo & Company (NYSE:WFC) Q3 2019 Earni...,4820,2019-10-15,Mortgage originations increased $5 billion fro...,NaN,John Shrewsberry,WFC,49.270000,16870500.0,50.110001,38478400.0


In [153]:
merged_df['percent_change_price'] = ((merged_df['next_close']/merged_df['base_close'])-1)*100
merged_df['percent_change_volume'] = ((merged_df['next_volume']/merged_df['base_volume'])-1)*100

merged_df

,call_datetime,base_date,base_day_of_week,call_date,call_section,call_time,call_title,id,next_date,sentence,sentiment,speaker,ticker,base_close,base_volume,next_close,next_volume,percent_change_price,percent_change_volume
0,2019-10-29 10:00:00,2019-10-28,0.0,2019-10-29,question,10:00:00,General Motors Company (NYSE:GM) Q3 2019 Earni...,1833,2019-10-29,Any update that you can share in terms of mile...,NaN,Itay Michaeli,GM,36.639999,10645500.0,38.209999,18562500.0,4.284934,74.369452
1,2019-10-24 08:00:00,2019-10-23,2.0,2019-10-24,answer,08:00:00,"Twitter, Inc. (NYSE:TWTR) Q3 2019 Results Earn...",3984,2019-10-24,Hope that helps.,NaN,Ned Segal,TWTR,38.830002,20276200.0,30.750000,105112500.0,-20.808657,418.403350
2,2019-10-24 08:00:00,2019-10-23,2.0,2019-10-24,answer,08:00:00,"Twitter, Inc. (NYSE:TWTR) Q3 2019 Results Earn...",3929,2019-10-24,Since you're probably wondering and we've talk...,NaN,Ned Segal,TWTR,38.830002,20276200.0,30.750000,105112500.0,-20.808657,418.403350
3,2019-10-30 08:00:00,2019-10-29,1.0,2019-10-30,presentation,08:00:00,General Electric Company (NYSE:GE) Q3 2019 Ear...,1004,2019-10-30,Good morning and welcome to GE's third quarter...,NaN,Steve Winoker,GE,9.070000,45759000.0,10.110000,215943400.0,11.466373,371.914596
4,2019-10-29 10:00:00,2019-10-28,0.0,2019-10-29,answer,10:00:00,General Motors Company (NYSE:GM) Q3 2019 Earni...,2010,2019-10-29,There's no other like one time item or whateve...,NaN,Dhivya Suryadevara,GM,36.639999,10645500.0,38.209999,18562500.0,4.284934,74.369452
5,2019-10-30 08:00:00,2019-10-29,1.0,2019-10-30,presentation,08:00:00,General Electric Company (NYSE:GE) Q3 2019 Ear...,1042,2019-10-30,"We started putting that cash to work, with a $...",NaN,Larry Culp,GE,9.070000,45759000.0,10.110000,215943400.0,11.466373,371.914596
6,2019-10-15 11:00:00,2019-10-14,0.0,2019-10-15,presentation,11:00:00,Wells Fargo & Company (NYSE:WFC) Q3 2019 Earni...,4837,2019-10-15,We have 30.2 million digital active customers ...,NaN,John Shrewsberry,WFC,49.270000,16870500.0,50.110001,38478400.0,1.704892,128.080970
7,2019-10-30 08:00:00,2019-10-29,1.0,2019-10-30,answer,08:00:00,General Electric Company (NYSE:GE) Q3 2019 Ear...,1563,2019-10-30,"So, I think what you see a little bit relative...",NaN,Larry Culp,GE,9.070000,45759000.0,10.110000,215943400.0,11.466373,371.914596
8,2019-10-23 09:00:00,2019-10-22,1.0,2019-10-23,answer,09:00:00,Invesco Ltd. (NYSE:IVZ) Q3 2019 Earnings Confe...,2226,2019-10-23,I think it is very hard to provide guidance on...,NaN,Loren Starr,IVZ,16.030001,8068200.0,17.280001,9388100.0,7.797879,16.359287
9,2019-10-15 11:00:00,2019-10-14,0.0,2019-10-15,presentation,11:00:00,Wells Fargo & Company (NYSE:WFC) Q3 2019 Earni...,4820,2019-10-15,Mortgage originations increased $5 billion fro...,NaN,John Shrewsberry,WFC,49.270000,16870500.0,50.110001,38478400.0,1.704892,128.080970
